# Your First AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) from Oxford of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load the image dataset and create a pipeline.
* Build and Train an image classifier on this dataset.
* Use your trained model to perform inference on flower images.

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

## Import Resources

In [ ]:
# The new version of dataset is only available in the tfds-nightly package.
# %pip --no-cache-dir install tfds-nightly --user
# DON'T MISS TO RESTART THE KERNEL

In [1]:
# TODO: Make all necessary imports.
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import json

from PIL import Image

import matplotlib.pyplot as plt

## Load the Dataset

Here you'll use `tensorflow_datasets` to load the [Oxford Flowers 102 dataset](https://www.tensorflow.org/datasets/catalog/oxford_flowers102). This dataset has 3 splits: `'train'`, `'test'`, and `'validation'`.  You'll also need to make sure the training data is normalized and resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet, but you'll still need to normalize and resize the images to the appropriate size.

In [2]:
# TODO: Load the dataset with TensorFlow Datasets.
ds, ds_info = tfds.load('oxford_flowers102', with_info=True, as_supervised=True)

# TODO: Create a training set, a validation set and a test set.
train_data, test_data, validate_data = ds['train'], ds['test'], ds['validation']

In [3]:
ds_info

tfds.core.DatasetInfo(
    name='oxford_flowers102',
    version=2.1.1,
    description='The Oxford Flowers 102 dataset is a consistent of 102 flower categories commonly occurring
in the United Kingdom. Each class consists of between 40 and 258 images. The images have
large scale, pose and light variations. In addition, there are categories that have large
variations within the category and several very similar categories.

The dataset is divided into a training set, a validation set and a test set.
The training set and validation set each consist of 10 images per class (totalling 1020 images each).
The test set consists of the remaining 6149 images (minimum 20 per class).',
    homepage='https://www.robots.ox.ac.uk/~vgg/data/flowers/102/',
    features=FeaturesDict({
        'file_name': Text(shape=(), dtype=tf.string),
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=102),
    }),
    total_num_examples=8

## Explore the Dataset

In [4]:
# TODO: Get the number of examples in each set from the dataset info.
num_ex_train = ds_info.splits['train'].num_examples
num_ex_test = ds_info.splits['test'].num_examples
num_ex_validate = ds_info.splits['validation'].num_examples

# TODO: Get the number of classes in the dataset from the dataset info.
num_classes = ds_info.features['label'].num_classes

In [ ]:
# Extra cell for debugging
print(num_ex_train, num_ex_test, num_ex_validate, num_classes)
print(train_data)

In [ ]:
# TODO: Print the shape and corresponding label of 3 images in the training set.
for i, taken_image in enumerate (train_data.take(3)):
    label = taken_image[1].numpy() + 1
    shape = taken_image[0].shape
    print(f'Image {i+1} - Shape: {shape}, Label: {label}')


In [ ]:
# TODO: Plot 1 image from the training set. Set the title 
# of the plot to the corresponding image label. 
img = taken_image[0].numpy()
label_num = taken_image[1].numpy() + 1

plt.imshow(img)
plt.title(label_num)
plt.show()


### Label Mapping

You'll also need to load in a mapping from label to category name. You can find this in the file `label_map.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/3.7/library/json.html). This will give you a dictionary mapping the integer coded labels to the actual names of the flowers.

In [ ]:
with open('label_map.json', 'r') as f:
    class_names = json.load(f)

In [ ]:
# TODO: Plot 1 image from the training set. Set the title 
# of the plot to the corresponding class name. 
for taken_image in train_data.take(1):
    img = taken_image[0].numpy()
    label = taken_image[1].numpy() + 1
    title = class_names[str(label)]

    plt.imshow(img)
    plt.colorbar()
    plt.title(title)
    plt.show()

    print(f'Label {label} = {title.title()}')

## Create Pipeline

In [ ]:
# Much of this from Transfer Learning II lesson
BATCH_SIZE = 32
IMG_DIM = 224

def format_image(image, label):
#     print(tf.cast(label, tf.int16))
    label = tf.cast(label, tf.int16)
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (IMG_DIM, IMG_DIM))
    image /= 255.0
    return image, label

training_batches = train_data.shuffle(num_ex_train // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validate_data.map(format_image).batch(BATCH_SIZE).prefetch(1)
testing_batches = test_data.map(format_image).batch(BATCH_SIZE).prefetch(1)

In [ ]:
# # TODO: Create a pipeline for each set.
# # Most of this is from the Data Augmentation lesson

# # The stuff below here that has a single # all worked

# BATCH_SIZE = 32
# IMG_DIM = 224

# # This generator will be used on the train and test sets
# img_gen_train = ImageDataGenerator(rescale=1.0/255,
#                                    rotation_range=45,
#                                    width_shift_range=0.2,
#                                    height_shift_range=0.2,
#                                    shear_range=0.2,
#                                    horizontal_flip=True,
#                                    fill_mode='nearest')

# # train_data_gen = img_gen_train.flow

# img_gen_validate = ImageDataGenerator(rescale=1.0/255)

# X_train = []
# y_train = []
# for taken_image in train_data.take(-1):
#     X_train.append(taken_image[0].numpy())
#     y_train.append(taken_image[1].numpy() + 1)
    
# X_test = []
# y_test = []
# for taken_image in test_data.take(-1):
#     X_test.append(taken_image[0].numpy())
#     y_test.append(taken_image[1].numpy() + 1)
    
# X_valid = []
# y_valid = []
# for taken_image in validate_data.take(-1):
#     X_valid.append(taken_image[0].numpy())
#     y_valid.append(taken_image[1].numpy() + 1)

# # X_train = np.array(list([x[0].numpy() for x in list(train_data)]))
# # y_train = np.array([x[1].numpy() + 1 for x in list(train_data)])

# # I think I need to resize all images to be the same dimensions:
# for i in range(num_ex_train):
#     X_train[i] = tf.image.resize(X_train[i], (IMG_DIM, IMG_DIM)) # / 255.0

# # X_train.shape[0]
# # img_gen_train.fit(X_train)

# # X_train

# # img_gen_train.flow(X_train,
# #                    y_train,
# #                    batch_size=BATCH_SIZE,
# #                    shuffle=True)

# idx = 200

# plt.imshow(X_valid[idx] / 255)
# plt.title(class_names[str(y_valid[idx])])
# plt.show();
# print(y_valid[idx])

# # img_gen_train.fit(X_train)
# # train_data
# X_train[0] = np.expand_dims(X_train[0], 0)
# X_valid[0] = np.expand_dims(X_valid[0], 0)
# X_train[0].shape


# idx = 50

# plt.imshow(X_train[idx] / 255)
# plt.title(class_names[str(y_train[idx])])
# plt.show();
# print(y_train[idx])

# # X_train = X_train.astype('float32')
# # X_train = X_train.reshape((X_train.shape[0], IMG_DIM, IMG_DIM, 3))

# img_gen_train.fit(X_train[0])

# # img_gen_train.fit(X_train.reshape(num_ex_train, -1))

# train_data_gen = img_gen_train.flow(X_train[0], 
#                                     batch_size=BATCH_SIZE,
#                                     shuffle=True)

# valid_data_gen = img_gen_validate.flow(X_valid[0],
#                                        batch_size=BATCH_SIZE)

# Build and Train the Classifier

Now that the data is ready, it's time to build and train the classifier. You should use the MobileNet pre-trained model from TensorFlow Hub to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! 

Refer to the rubric for guidance on successfully completing this section. Things you'll need to do:

* Load the MobileNet pre-trained network from TensorFlow Hub.
* Define a new, untrained feed-forward network as a classifier.
* Train the classifier.
* Plot the loss and accuracy values achieved during training for the training and validation set.
* Save your trained model as a Keras model. 

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right.

**Note for Workspace users:** One important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module. Also, If your model is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [ ]:
# TODO: Build and train your network.
# Download an existing model - I'm using one for plants since I figure it is close to flowers already
# URL = 'https://tfhub.dev/google/aiy/vision/classifier/plants_V1/1'

URL = 'https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4'

feature_extractor = hub.KerasLayer(URL, input_shape=(IMG_DIM, IMG_DIM, 3))
feature_extractor.trainable = False

model = tf.keras.Sequential([
        feature_extractor,
        tf.keras.layers.Dense(102, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
print(f'GPU Available? :: {tf.test.is_gpu_available()}')

In [ ]:
EPOCHS = 20

history = model.fit(training_batches, epochs=EPOCHS)

In [ ]:
# TODO: Plot the loss and accuracy values achieved during training for the training and validation set.
for image_batch, label_batch in training_batches.take(1):
    loss, accuracy = model.evaluate(image_batch, label_batch)
    
print(f'Loss: {loss}, accuracy: {accuracy}')
print (history.history['accuracy'])

f, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=False)
ax1.plot(range(1, EPOCHS + 1), history.history['accuracy'])
ax1.plot([1, EPOCHS + 1], [0.85, 0.85], ':r')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Accuracy')
ax1.set_title('Accuracy vs Epoch')
ax1.grid()

ax2.plot(range(1, EPOCHS + 1), history.history['loss'])
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Loss')
ax2.set_title('Loss vs Epoch')
ax2.grid()
plt.show();

## Testing your Network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Print the loss and accuracy values achieved on the entire test set.
# for image_batch, label_batch in testing_batches.take(-1):
loss, accuracy = model.evaluate(testing_batches)#image_batch, label_batch)
print(f'Loss: {loss}, accuracy: {accuracy}')


## Save the Model

Now that your network is trained, save the model so you can load it later for making inference. In the cell below save your model as a Keras model (*i.e.* save it as an HDF5 file).

In [ ]:
# TODO: Save your trained model as a Keras model.
model.save('./flower_recog_model.h5')


## Load the Keras Model

Load the Keras model you saved above.

In [ ]:
# TODO: Load the Keras model
# NOTE: Found solution to KerasLayer error when loading model at https://github.com/tensorflow/tensorflow/issues/26835
loaded_model = tf.keras.models.load_model('./flower_recog_model.h5', custom_objects={'KerasLayer':hub.KerasLayer})

loaded_model.summary()

# Inference for Classification

Now you'll write a function that uses your trained network for inference. Write a function called `predict` that takes an image, a model, and then returns the top $K$ most likely class labels along with the probabilities. The function call should look like: 

```python
probs, classes = predict(image_path, model, top_k)
```

If `top_k=5` the output of the `predict` function should be something like this:

```python
probs, classes = predict(image_path, model, 5)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

Your `predict` function should use `PIL` to load the image from the given `image_path`. You can use the [Image.open](https://pillow.readthedocs.io/en/latest/reference/Image.html#PIL.Image.open) function to load the images. The `Image.open()` function returns an `Image` object. You can convert this `Image` object to a NumPy array by using the `np.asarray()` function.

The `predict` function will also need to handle pre-processing the input image such that it can be used by your model. We recommend you write a separate function called `process_image` that performs the pre-processing. You can then call the `process_image` function from the `predict` function. 

### Image Pre-processing

The `process_image` function should take in an image (in the form of a NumPy array) and return an image in the form of a NumPy array with shape `(224, 224, 3)`.

First, you should convert your image into a TensorFlow Tensor and then resize it to the appropriate size using `tf.image.resize`.

Second, the pixel values of the input images are typically encoded as integers in the range 0-255, but the model expects the pixel values to be floats in the range 0-1. Therefore, you'll also need to normalize the pixel values. 

Finally, convert your image back to a NumPy array using the `.numpy()` method.

In [ ]:
# TODO: Create the process_image function
IMG_DIM = 224

def process_image(np_image):
    tf_image = tf.convert_to_tensor(np_image)
    tf_image = tf.image.resize(np_image, [IMG_DIM, IMG_DIM])
    tf_image /= 255.0
    return tf_image.numpy()


To check your `process_image` function we have provided 4 images in the `./test_images/` folder:

* cautleya_spicata.jpg
* hard-leaved_pocket_orchid.jpg
* orange_dahlia.jpg
* wild_pansy.jpg

The code below loads one of the above images using `PIL` and plots the original image alongside the image produced by your `process_image` function. If your `process_image` function works, the plotted image should be the correct size. 

In [ ]:
from PIL import Image

image_path = './test_images/cautleya_spicata.jpg'
im = Image.open(image_path)
test_image = np.asarray(im)

processed_test_image = process_image(test_image)

fig, (ax1, ax2) = plt.subplots(figsize=(10,10), ncols=2)
ax1.imshow(test_image)
ax1.set_title('Original Image')
ax2.imshow(processed_test_image)
ax2.set_title('Processed Image')
plt.tight_layout()
plt.show()

Once you can get images in the correct format, it's time to write the `predict` function for making inference with your model.

### Inference

Remember, the `predict` function should take an image, a model, and then returns the top $K$ most likely class labels along with the probabilities. The function call should look like: 

```python
probs, classes = predict(image_path, model, top_k)
```

If `top_k=5` the output of the `predict` function should be something like this:

```python
probs, classes = predict(image_path, model, 5)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

Your `predict` function should use `PIL` to load the image from the given `image_path`. You can use the [Image.open](https://pillow.readthedocs.io/en/latest/reference/Image.html#PIL.Image.open) function to load the images. The `Image.open()` function returns an `Image` object. You can convert this `Image` object to a NumPy array by using the `np.asarray()` function.

**Note:** The image returned by the `process_image` function is a NumPy array with shape `(224, 224, 3)` but the model expects the input images to be of shape `(1, 224, 224, 3)`. This extra dimension represents the batch size. We suggest you use the `np.expand_dims()` function to add the extra dimension. 

In [ ]:
# TODO: Create the predict function
def predict(image_path, model, top_k):
    image = process_image(np.asarray(Image.open(image_path)))
    image = np.expand_dims(image, axis=0)
    
    pred = model.predict(image)
    
    pred = pred[0].tolist()
    
    probs, classes = tf.math.top_k(pred, k=top_k)
    
    return probs, classes + 1 # Correct class index to 1-based used in class_names

In [ ]:
test_image = './test_images/wild_pansy.jpg'

probs, classes = predict(test_image, loaded_model, 5)
print(probs)
print(classes)
print(class_names[str(classes.numpy()[0])])

# Sanity Check

It's always good to check the predictions made by your model to make sure they are correct. To check your predictions we have provided 4 images in the `./test_images/` folder:

* cautleya_spicata.jpg
* hard-leaved_pocket_orchid.jpg
* orange_dahlia.jpg
* wild_pansy.jpg

In the cell below use `matplotlib` to plot the input image alongside the probabilities for the top 5 classes predicted by your model. Plot the probabilities as a bar graph. The plot should look like this:

<img src='assets/inference_example.png' width=600px>

You can convert from the class integer labels to actual flower names using `class_names`. 

In [ ]:
# TODO: Plot the input image long with the top 5 classes
categories = []
for flower_class in classes.numpy():
    categories.append(class_names[str(flower_class)])
    
print(categories)
print(sorted(probs.numpy().tolist()))

fig, (ax1, ax2) = plt.subplots(figsize=(12, 6), ncols=2)
ax1.imshow(Image.open(test_image))
# ax1.set_title('Input Test Image')
ax2.barh(categories, probs) #sorted(probs.numpy().tolist()))
ax2.invert_yaxis()
ax2.set_title('Class Probability')
ax2.set_xlim([0.0, 1.0])
plt.tight_layout()
plt.show()